
# Prerequisite

In [ ]:

#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Code, Dataset
from azure.identity import InteractiveBrowserCredential

In [ ]:
#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
# get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# Basic pipeline job

## Build pipeline

In [ ]:
from azure.ml import dsl
from azure.ml.dsl import Pipeline
from azure.ml.entities import Component
from pathlib import Path

parent_dir = ''

# 1. Get component definition and transfer component to functions
tf_component = Component.load(path=parent_dir + "./component.yml")
tf_func = dsl.load_component(component=tf_component)

# 2. Construct pipeline
@dsl.pipeline(
    description="Train using TF component",
)
def sample_pipeline():
    tf_job = tf_func(epochs=1)
    tf_job.compute = "cpu-cluster"
    tf_job.outputs.trained_model_output.mode = "upload"
    tf_job.distribution.worker_count = 1
    tf_job.distribution.distributionType = "tensorflow"
    tf_job.resources.instance_count = 2

# create pipeline instance
pipeline = sample_pipeline()

# Submit pipeline job

In [ ]:
# submit job to workspace
returned_job = ml_client.jobs.create_or_update(pipeline, experiment_name="tf_mnist", continue_run_on_step_failure=True)
returned_job.services["Studio"].endpoint